In [4]:
#调试存储方式，研究如何存储数据

In [ ]:
#研究这个代码存储问题，解决和相应的geturl问题

In [29]:
import requests,json,time,re
from requests.exceptions import RequestException
from pyquery import PyQuery as pq
from fontTools.ttLib import TTFont
from io import BytesIO
import pandas as pd

 
start_url = 'https://www.qidian.com/finish?action=hidden&orderId=&style=1&pageSize=20&siteid=1&pubflag=0&hiddenField=2&page=1'
 
def get_font(url):
    response = requests.get(url)
    font = TTFont(BytesIO(response.content))
    cmap = font.getBestCmap()
    font.close()
    return cmap
 
def get_encode(cmap,values):
    WORD_MAP = {'zero':'0','one':'1','two':'2','three':'3','four':'4','five':'5','six':'6','seven':'7','eight':'8','nine':'9','period':'.'}
    word_count=''
    for value in values.split(';'):
        value = value[2:]
        key = cmap[int(value)]
        word_count += WORD_MAP[key]
    return word_count
 
def get_index(start_url):
    #获取当前页面的html
    response = requests.get(start_url).text   
    doc = pq(response)
    #获取当前字体文件名称
    classattr = doc('p.update > span > span').attr('class')
    pattern = '</style><span.*?%s.*?>(.*?)</span>'%classattr
    #获取当前页面所有被字数字符
    numberlist = re.findall(pattern,response)
    #获取当前包含字体文件链接的文本
    fonturl = doc('p.update > span > style').text() 
    #通过正则获取当前页面字体文件链接
    url = re.search('woff.*?url.*?\'(.+?)\'.*?truetype',fonturl) .group(1)
    cmap = get_font(url)
    books = doc('.all-img-list li').items()
    i = 0
    df = pd.DataFrame()
    for book in books:
        item = {}
        item['bookname'] = book('.book-mid-info h4 a').text()
        item['author'] = book('.name').text()
        item['classes'] = book('p.author > a:nth-child(4)').text()
        item['content'] = book('.intro').text()
        item['number'] = get_encode(cmap,numberlist[i][:-1])
        i += 1
        df = df.append(item, ignore_index=True)
    return df

url = start_url
get_index(url)

,author,bookname,classes,content,number
0,卧牛真人,修真四万年,科幻,“倘若这宇宙，真是一片残酷血腥的黑暗森林，我们修真者，也会燃烧自己的生命，绽放出微弱的火花！...,1040.22
1,贱宗首席弟子,大魏宫廷,历史,生作大魏皇子，愿当盛世闲王。志在偎红倚翠犬马声色，胸怀家国百姓社稷安危。若兄贤，若弟明，尔为...,674.68
2,王梓钧,民国之文豪崛起,历史,周赫煊回到民国时代，不会带兵打仗，不懂科学技术，不喜从政从商，那他还能干什么？唉，抄书抄诗混...,249.66
3,寒门 二次元,奶爸的文艺人生,都市,“粑粑！”刚穿越到平行世界，就看到有个精致可爱的小女孩喊自己爸爸，杨轶表示有些懵逼。好吧，曦...,315.06
4,二将,国际制造商,都市,韩义在路上扫了个二维码，然后桌面上多了个应用。这个应用很奇怪，它可以捕捉到现实里的一切物品，...,216.12
5,卖报小郎君,我的姐姐是大明星,都市,某年某月某天，姐姐忽然说：“我要做明星。”秦泽淡定道：“老姐，你想被老爹活活打死，我不反对。...,215.96
6,孤单地飞,神道丹尊,玄幻,绝世强者、一代丹帝凌寒为追求成神之路而殒落，万年后携《不灭天经》重生于同名少年，从此风云涌动...,1019.58
7,蝴蝶蓝,全职高手,游戏,网游荣耀中被誉为教科书级别的顶尖高手，因为种种原因遭到俱乐部的驱逐，离开职业圈的他寄身于一家...,535.02
8,白色十三号,最佳影星,都市,新书《魔气熏天》已发布我要躺在钞票堆里打滚！我要做好莱坞巨星！群：323320648。,311.45
9,忘语,凡人修仙传,仙侠,一个普通山村小子，偶然下进入到当地江湖小门派，成了一名记名弟子。他以这样身份，如何在门派中立...,744.75


In [10]:
# problem is here!
# where is mongo?
# where is my data?

In [13]:
import pandas as pd
di = {'bookname': '修真四万年',
  'author': '卧牛真人',
  'classes': '科幻',
  'content': '“倘若这宇宙，真是一片残酷血腥的黑暗森林，我们修真者，也会燃烧自己的生命，绽放出微弱的火花！”“哪怕这火花再微弱，再短暂，再渺小，可是只要我们源源不断，前赴后继，终有一日，火花会点燃杂草，杂草会燎到灌',
  'number': '1040.22'}

In [15]:
df = pd.DataFrame(di,index=[0])

In [16]:
df

,bookname,author,classes,content,number
0,修真四万年,卧牛真人,科幻,“倘若这宇宙，真是一片残酷血腥的黑暗森林，我们修真者，也会燃烧自己的生命，绽放出微弱的火花！...,1040.22


In [17]:
d1 = {'bookname': '大魏宫廷',
  'author': '贱宗首席弟子',
  'classes': '历史',
  'content': '生作大魏皇子，愿当盛世闲王。志在偎红倚翠犬马声色，胸怀家国百姓社稷安危。若兄贤，若弟明，尔为人王吾偷闲。若尔不能使国强，吾来登基做帝王！————弘润《你不行我上》',
  'number': '674.68'}

In [18]:
df1 =  pd.DataFrame(d1,index=[0])

In [19]:
df1

,bookname,author,classes,content,number
0,大魏宫廷,贱宗首席弟子,历史,生作大魏皇子，愿当盛世闲王。志在偎红倚翠犬马声色，胸怀家国百姓社稷安危。若兄贤，若弟明，尔为...,674.68


In [23]:
result = df1.append(di, ignore_index=True)

In [24]:
result

,bookname,author,classes,content,number
0,大魏宫廷,贱宗首席弟子,历史,生作大魏皇子，愿当盛世闲王。志在偎红倚翠犬马声色，胸怀家国百姓社稷安危。若兄贤，若弟明，尔为...,674.68
1,修真四万年,卧牛真人,科幻,“倘若这宇宙，真是一片残酷血腥的黑暗森林，我们修真者，也会燃烧自己的生命，绽放出微弱的火花！...,1040.22


In [25]:
df2 = pd.DataFrame()

In [26]:
df2.append(di, ignore_index=True)

,author,bookname,classes,content,number
0,卧牛真人,修真四万年,科幻,“倘若这宇宙，真是一片残酷血腥的黑暗森林，我们修真者，也会燃烧自己的生命，绽放出微弱的火花！...,1040.22


In [ ]:
#方法有问题